In [1]:
from stage1_models import *
from stage2_models import *
from conflict_utils import *
from key import *

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Library/Frameworks/Python.framework/V

In [2]:
#Load the dataset with all the TPM features

train_tpm = unpickle_embeddings("/Users/priyadcosta/Documents/GitHub/coefficientofconflict/sandbox/embeddings/initial_inputs/tpm_hand_labeled.pickle") #Hand labeled data
test_tpm = unpickle_embeddings("/Users/priyadcosta/Documents/GitHub/coefficientofconflict/sandbox/embeddings/initial_inputs/tpm_non_hand_labeled.pickle") 

In [3]:
output_tpm = classify_using_tpm_with_logreg(train_tpm,test_tpm)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://sciki

Top 5 predictive features for d_content_average:
hasnegative: -0.7662223999938567
direct_question: -0.628093288335286
factuality: 0.5694172100797291
2nd_person: -0.4901181692589242
1st_person_start: 0.45655191016250146


Top 5 predictive features for d_expression_average:
2nd_person: -0.8379396812527075
subjunctive: -0.6649044490741922
1st_person_start: 0.5935545169822981
hasnegative: -0.5646761284081977
factuality: 0.49594868929898517


Top 5 predictive features for oi_content_average:
hasnegative: -0.9430988090596286
factuality: 0.5494432955992082
haspositive: 0.5105106641893447
hedges: 0.4985978870418739
2nd_person: -0.47503263695182923


Top 5 predictive features for oi_expression_average:
hashedge: -0.7115748203444149
1st_person: 0.543783366792791
1st_person_start: -0.2911873062025171
direct_start: 0.26849839493837624
1st_person_pl: 0.26821377075842884




In [4]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import plotly.express as px

def classify_using_tpm_with_pca(train, test, features):
    targets = ['d_content_average', 'd_expression_average', 'oi_content_average', 'oi_expression_average']
    
    # Fill missing values with 0.0
    train.fillna(0.0, inplace=True)
    test.fillna(0.0, inplace=True)
    
    # Binary conversion of the target variables
    train[targets] = (train[targets] > 0.5).astype(int)
    
    # Scale the data
    scaler = StandardScaler()
    train_scaled = scaler.fit_transform(train[features])
    test_scaled = scaler.transform(test[features])
    
    # Fit PCA
    pca = PCA(n_components=5)
    pca.fit(train_scaled)
    
    # Visualize the contributions of the original features to the first two PCA components
    components_df = pd.DataFrame(pca.components_.T, columns=[f'PC{i+1}' for i in range(5)], index=features)
    fig = px.scatter(components_df, x='PC1', y='PC2', text=components_df.index, 
                     title="Feature Contributions")
    fig.update_traces(textposition='top center')
    fig.update_layout(height=800, width=800)
    fig.show()
    
        
    # Visualize the contributions of the original features to the first two PCA components
    components_df = pd.DataFrame(pca.components_.T, columns=[f'PC{i+1}' for i in range(5)], index=features)
    fig = px.scatter(components_df, x='PC3', y='PC4', text=components_df.index, 
                     title="Feature Contributions")
    fig.update_traces(textposition='top center')
    fig.update_layout(height=800, width=800)
    fig.show()

    # Define and fit the full pipeline including classification
    full_pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=5)),
        ('classifier', MultiOutputClassifier(LogisticRegression(solver='saga', max_iter=1000)))
    ])
    full_pipeline.fit(train[features], train[targets])
    
    # Predict on the test set
    y_pred = full_pipeline.predict(test[features])
    
    # Append the predictions to the test DataFrame
    y_pred_df = pd.DataFrame(y_pred, columns=targets, index=test.index)
    pred_df = pd.concat([test, y_pred_df], axis=1)

    # Optionally, print explained variance ratio by PCA components
    print("Explained variance ratio by PCA components:", pca.explained_variance_ratio_)
    
    return pred_df


In [5]:
tpm_all_dim = classify_using_tpm_with_pca(train_tpm, test_tpm, ALL_FEATURES)

Explained variance ratio by PCA components: [0.24657041 0.04330479 0.03447293 0.03216725 0.028926  ]


In [6]:
tpm_full_dim = classify_using_tpm_with_logreg(train_tpm,test_tpm)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_

Top 5 predictive features for d_content_average:
hasnegative: -0.7662223999938567
direct_question: -0.628093288335286
factuality: 0.5694172100797291
2nd_person: -0.4901181692589242
1st_person_start: 0.45655191016250146


Top 5 predictive features for d_expression_average:
2nd_person: -0.8379396812527075
subjunctive: -0.6649044490741922
1st_person_start: 0.5935545169822981
hasnegative: -0.5646761284081977
factuality: 0.49594868929898517


Top 5 predictive features for oi_content_average:
hasnegative: -0.9430988090596286
factuality: 0.5494432955992082
haspositive: 0.5105106641893447
hedges: 0.4985978870418739
2nd_person: -0.47503263695182923


Top 5 predictive features for oi_expression_average:
hashedge: -0.7115748203444149
1st_person: 0.543783366792791
1st_person_start: -0.2911873062025171
direct_start: 0.26849839493837624
1st_person_pl: 0.26821377075842884




In [7]:
# get_stage1_label_counts(tpm_out_put_reduced_dim)

In [8]:
get_stage1_label_counts(tpm_full_dim)

Counts of 1's:
{'d_content_average': 0.63, 'd_expression_average': 0.59, 'oi_content_average': 0.61, 'oi_expression_average': 0.43, 'dataset_numeric': 0.85}

Counts of 0's:
{'d_content_average': 0.37, 'd_expression_average': 0.41, 'oi_content_average': 0.39, 'oi_expression_average': 0.57, 'dataset_numeric': 0.15}


In [9]:
embeddings = unpickle_embeddings("../sandbox/embeddings/stage_1_outputs/stage1_embeddings_output.pickle")
get_stage1_label_counts(embeddings)

Counts of 1's:
{'d_content_average': 0.99, 'd_expression_average': 0.98, 'oi_content_average': 0.98, 'oi_expression_average': 0.0, 'dataset_numeric': 0.43}

Counts of 0's:
{'d_content_average': 0.0, 'd_expression_average': 0.01, 'oi_content_average': 0.01, 'oi_expression_average': 0.99, 'dataset_numeric': 0.56}


In [10]:
# Dataset hand-labeled by RAs
original_hand_labeled = unpickle_embeddings("../sandbox/embeddings/initial_inputs/embeddings_hand_labeled.pickle")
get_stage1_label_counts(original_hand_labeled)

Counts of 1's:
{'d_content_average': 0.68, 'd_expression_average': 0.53, 'oi_content_average': 0.56, 'oi_expression_average': 0.04, 'dataset_numeric': 0.38}

Counts of 0's:
{'d_content_average': 0.27, 'd_expression_average': 0.29, 'oi_content_average': 0.29, 'oi_expression_average': 0.79, 'dataset_numeric': 0.62}


### Reduce dimensions
### Try XGBoost, MLP
### Relax binary assumption
### LR with L1,L2 penalty
### Sample Weighting

In [11]:
classify_using_tpm_with_nn(train_tpm, test_tpm, TOP_FEATURES)
classify_using_tpm_with_xgboost(train_tpm, test_tpm, TOP_FEATURES)
classify_using_tpm_with_decision_tree(train_tpm, test_tpm, TOP_FEATURES)

Feature importances:
d_content_average:
sadness_lexical_per_100: 0.04499288275837898
second_person_lexical_per_100: 0.061536308377981186
positive_words_lexical_per_100: 0.04236001521348953
first_person_lexical_per_100: 0.03622331842780113
hedge_words_lexical_per_100: 0.08050639182329178
num_question_naive: 0.049231041222810745
word_TTR: 0.03439102694392204
first_pronouns_proportion: 0.04312300682067871
mimicry_bert: 0.04528135806322098
moving_mimicry: 0.04617634043097496
please: 0.0
hashedge: 0.021969936788082123
indirect_btw: 0.0
hedges: 0.04755619913339615
factuality: 0.0387289933860302
gratitude: 0.0
apologizing: 0.020720643922686577
1st_person_pl: 0.052855681627988815
1st_person: 0.03178779035806656
2nd_person: 0.021943124011158943
direct_question: 0.08855170756578445
direct_start: 0.04874487221240997
haspositive: 0.04858361929655075
hasnegative: 0.09473569691181183


d_expression_average:
sadness_lexical_per_100: 0.07559312134981155
second_person_lexical_per_100: 0.044345930218696

,Unnamed0,conversation_num,message,speaker_nickname,id,timestamp,metascore,reply_to,conversation_length,dataset_numeric,...,haspositive,hasnegative,subjunctive,indicative,forward_flow,certainty_rocklage,d_content_average,d_expression_average,oi_content_average,oi_expression_average
0,13691,1019_A,the actual leaders of bailedout firms should b...,Rimfish,t3_292cja,0.000000e+00,0.0,ORIGINAL_POST,5,0,...,1,1,0,0,0.000000,5.647778,1,1,0,0
1,13692,1019_A,its not business leaders who keep the profits ...,Thoguth,t1_cigrpj8_1,1.403717e+09,2.0,t3_292cja,5,0,...,1,1,0,0,0.496333,5.753333,1,1,1,0
2,13693,1019_A,so theres really not that much of a moral haz...,Thoguth,t1_cigrpj8_2,1.403717e+09,2.0,t3_292cja,5,0,...,1,1,0,0,0.519183,5.420000,1,1,1,0
3,13694,1019_A,on the other hand the shareholder of a failing...,Thoguth,t1_cigrpj8_3,1.403717e+09,2.0,t3_292cja,5,0,...,1,1,0,0,0.679783,5.450000,1,1,0,1
4,13695,1019_A,business owners are often also shareholders wh...,Rimfish,t1_cigsdsm,1.403718e+09,0.0,t1_cigrpj8,5,0,...,1,0,0,0,0.411667,6.626667,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76884,129391,e8r47cx,you dont think a culture of murder theft rape ...,adidasbdd,e8rcogi,1.540945e+09,-1.0,e8rabip,7,1,...,0,1,0,0,0.612434,2.530000,1,0,0,0
76885,129392,e8r47cx,i think that doesnt exist in america,maybe_bait,e8rdfca,1.540945e+09,-2.0,e8rcogi,7,1,...,0,0,0,0,0.843804,4.610000,1,1,1,0
76886,129393,e8r47cx,the vast majority of the rapes murders thefts ...,adidasbdd,e8rdksn,1.540945e+09,0.0,e8rdfca,7,1,...,0,1,0,0,0.487119,7.505000,1,0,1,0
76887,129394,e8r47cx,society is forged by a coalition of people may...,maybe_bait,e8rk76w,1.540951e+09,-1.0,e8rdksn,7,1,...,0,1,0,0,0.555410,3.790000,1,1,1,0


In [12]:
# train_embeddings = unpickle_embeddings("../sandbox/embeddings/initial_inputs/embeddings_non_hand_labeled.pickle")
# reduce_embeddings_dimensionality(train_embeddings)

In [13]:
remove_quotes_from_dataset(train_tpm,'message')